如何使用单个GPU，然后是如何使用多个GPU和多个服务器（具有多个GPU）

在PyTorch中，每个数组都有一个设备（device）， 我们通常将其称为环境（context）。  
默认情况下，所有变量和相关的计算都分配给CPU。有时环境可能是GPU。  
当我们跨多个服务器部署作业时，事情会变得更加棘手。  
通过智能地将数组分配给环境， 我们可以最大限度地减少在设备之间传输数据的时间。  
例如，当在带有GPU的服务器上训练神经网络时， 我们通常希望模型的参数在GPU上。

要运行此部分中的程序，至少需要两个GPU。  
注意，对大多数桌面计算机来说，这可能是奢侈的，但在云中很容易获得。  
例如可以使用AWS EC2的多GPU实例。  
本书的其他章节大都不需要多个GPU， 而本节只是为了展示数据如何在不同的设备之间传递。

计算设备
我们可以指定用于存储和计算的设备，如CPU和GPU。  
默认情况下，张量是在内存中创建的，然后使用CPU计算它。

在PyTorch中，CPU和GPU可以用torch.device('cpu') 和torch.device('cuda')表示。  
应该注意的是，cpu设备意味着所有物理CPU和内存， 这意味着PyTorch的计算将尝试使用所有CPU核心。  
然而，gpu设备只代表一个卡和相应的显存。  
如果有多个GPU，我们使用torch.device(f'cuda:{i}') 来表示第i块GPU（从0开始）。  
另外，cuda:0和cuda是等价的。

In [1]:
import torch
from torch import nn

torch.device('cpu'), torch.device('cuda'), torch.device('cuda:1')

(device(type='cpu'), device(type='cuda'), device(type='cuda', index=1))

In [2]:
#查询可用gpu的数量
torch.cuda.device_count()

1

现在我们定义了两个方便的函数， 这两个函数允许我们在不存在所需所有GPU的情况下运行代码。

In [ ]:
def try_gpu(i=0):  #@save
    """如果存在，则返回gpu(i)，否则返回cpu()"""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

def try_all_gpus():  #@save
    """返回所有可用的GPU，如果没有GPU，则返回[cpu(),]"""
    devices = [torch.device(f'cuda:{i}') for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]

try_gpu(), try_gpu(10), try_all_gpus()


(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0)])

张量与GPU  
可以查询张量所在的设备。 默认情况下，张量是在CPU上创建的

In [4]:
x = torch.tensor([1, 2, 3])
x.device


device(type='cpu')

无论何时我们要对多个项进行操作， 它们都必须在同一个设备上。  
例如，如果我们对两个张量求和， 我们需要确保两个张量都位于同一个设备上， 否则框架将不知道在哪里存储结果，甚至不知道在哪里执行计算。

存储在GPU上  
有几种方法可以在GPU上存储张量。  
例如，我们可以在创建张量时指定存储设备。 
接下来，我们在第一个gpu上创建张量变量X。  
在GPU上创建的张量只消耗这个GPU的显存。  
我们可以使用nvidia-smi命令查看显存使用情况。  
一般来说，我们需要确保不创建超过GPU显存限制的数据。

In [6]:
X = torch.ones(2, 3, device=try_gpu())
X

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')

In [9]:
Y = torch.rand(2, 3, device=try_gpu(0))#这里如果有第二块GPU，try_gpu(1)
Y

tensor([[0.7178, 0.2061, 0.2288],
        [0.7464, 0.2408, 0.0329]], device='cuda:0')

复制  
如果我们要计算X + Y，我们需要决定在哪里执行这个操作。  
例如，我们可以将X传输到第二个GPU并在那里执行操作。  
不要简单地X加上Y，因为这会导致异常， 运行时引擎不知道该怎么做：它在同一设备上找不到数据会导致失败。  
由于Y位于第二个GPU上，所以我们需要将X移到那里， 然后才能执行相加运算。

In [10]:
Z = X.cuda(0)
print(X)
print(Z)


tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')
tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')


现在数据在同一个GPU上（Z和Y都在），我们可以将它们相加

In [11]:
Y + Z

tensor([[1.7178, 1.2061, 1.2288],
        [1.7464, 1.2408, 1.0329]], device='cuda:0')

假设变量Z已经存在于第二个GPU上。 如果我们还是调用Z.cuda(1)会发生什么？ 它将返回Z，而不会复制并分配新内存。

In [13]:
Z.cuda(0) is Z

True

神经网络与GPU  

类似地，神经网络模型可以指定设备。 下面的代码将模型参数放在GPU上

In [14]:
net = nn.Sequential(nn.Linear(3, 1))
net = net.to(device=try_gpu())

在接下来的几章中， 我们将看到更多关于如何在GPU上运行模型的例子， 因为它们将变得更加计算密集。

当输入为GPU上的张量时，模型将在同一GPU上计算结果

In [15]:
net(X)

tensor([[-1.2394],
        [-1.2394]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [16]:
#让我们确认模型参数存储在同一个GPU
net[0].weight.data.device


device(type='cuda', index=0)

总之，只要所有的数据和参数都在同一个设备上， 我们就可以有效地学习模型。 在下面的章节中，我们将看到几个这样的例子。